# VSB using AE (Anamoly detection)

Connecting colab to google drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Extracting data to a specific folder.

In [0]:
import zipfile
zip_ref = zipfile.ZipFile("/content/preprocessed.zip", 'r')
zip_ref.extractall("/content/data1")
zip_ref.close()

In [0]:
import os
a=os.listdir('/content/data1')

Since initially we don't know the index locations of the classes(0 & 1), we will be distinguishing the images corresponding to class 1 only.
* To do this, we will first generate a list of strings which match with the image names we are trying to find out the index locations of.
* We then run a loop and if a match is found between the real directory containing images and the list previously created, we will be collecting index locations of the image names corresponding to class 1.



In [0]:
#Step 1
list1=[]
for i in range(0,580):
    if i%3==0:
        list1.append('sample_1_'+str(i)+'.png')

#Step 2        
list2=[]
for i in range(0,len(a)):
      if a[i] in list1:
        list2.append(i)

In [5]:
len(list2)

194

Importing all necessary API's

In [6]:
# For controlling Python warnings - https://docs.python.org/3.6/library/warnings.html
import warnings
warnings.filterwarnings("ignore")

# For iterators - https://docs.python.org/3.1/library/itertools.html
import itertools

# For loading and handling data
import numpy as np
import pandas as pd

# For Normalization
from sklearn.preprocessing import MinMaxScaler

# For constructing DNN
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
from keras.layers import Input,Dropout,Dense 
from keras.models import Model
from keras import regularizers
from keras.utils.data_utils import get_file

# For visualization
import matplotlib.pyplot as plt

# For evaluating the accuracy
from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,precision_score,f1_score
%matplotlib inline
import os
from PIL import Image

Using TensorFlow backend.


Loading data

In [0]:
from numpy import *
path1='/content/data1'
imlist=os.listdir(path1)
num_samples=size(imlist)

In [8]:
num_samples

2904

Converting data into a matrix of vectors.

In [0]:
immatrix=array([array(Image.open('/content/data1' + '/' + im2)).flatten() for im2 in imlist], 'f')

In [10]:
len(immatrix)

2904

Creating an array of size equal to the row size of the data with initial values 1. We will then use the index locations which we found out earlier to change these label values to either 0 (if index of the new loop does not match with label) or 1 (if index of the new loop matches with the label)

In [0]:
labels=np.ones((num_samples,), dtype=int)

In [0]:
for i in range(0,len(labels)):
  if i in list2:
    labels[i]=1
  else:
    labels[i]=0

Normalizing data between 0 and 1. And then creating a dataframe out of it.

In [0]:
new1=((immatrix-immatrix.min())/(immatrix.max()-immatrix.min()))
df1=pd.DataFrame(new1,index=new1[:,0])

Adding a new column(the labels array which we worked with earlier) to the dataframe.

In [0]:
defi=labels.tolist()
df1['class']=labels

In [20]:
df1.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,65497,65498,65499,65500,65501,65502,65503,65504,65505,65506,65507,65508,65509,65510,65511,65512,65513,65514,65515,65516,65517,65518,65519,65520,65521,65522,65523,65524,65525,65526,65527,65528,65529,65530,65531,65532,65533,65534,65535,class
1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0
1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0


Splitting data into Train and test.

In [0]:
from sklearn.model_selection import train_test_split
X_train,X_test=train_test_split(df1,test_size=0.2,random_state=21)

Since we are dealing with anamoly detection, we will only feed 'normal' data into our model. In this case, normal would mean class 0. We do this so that during testing, any abnormal data would be classified as class 1.

In [0]:
X_train=X_train[X_train['class']==0]
X_train=X_train.drop('class',axis=1)
y_test=X_test['class']
X_test=X_test.drop('class',axis=1)

X_train=X_train.values
X_test=X_test.values

We create y_train which has 0's only.

In [0]:
y_train=np.ones((2165,), dtype=int)
y_train[:]=0

Training model.

In [115]:
def getModel():
    inp = Input(shape=(X_train.shape[1],))
    d1=Dropout(0.5)(inp)
    encoded = Dense(8, activation='relu', activity_regularizer=regularizers.l2(10e-3))(d1)
    decoded = Dense(X_train.shape[1], activation='relu')(encoded)
    autoencoder = Model(inp, decoded)
    autoencoder.compile(optimizer='adam', loss='mean_squared_error')
    return autoencoder

# Where y0=0 --> Fit only on normal network connections - A kind of one class classifier
autoencoder=getModel()
history=autoencoder.fit(X_train[np.where(y_train==0)],X_train[np.where(y_train==0)], 
               epochs=10,
                batch_size=100,
                shuffle=True,
                validation_split=0.1)

Train on 1948 samples, validate on 217 samples
Epoch 1/10
1948/1948 [==============================] - 5s 2ms/step - loss: 1.3177 - val_loss: 0.8299
Epoch 2/10
1948/1948 [==============================] - 4s 2ms/step - loss: 0.8150 - val_loss: 0.7959
Epoch 3/10
1948/1948 [==============================] - 4s 2ms/step - loss: 0.7814 - val_loss: 0.7630
Epoch 4/10
1948/1948 [==============================] - 4s 2ms/step - loss: 0.7490 - val_loss: 0.7312
Epoch 5/10
1948/1948 [==============================] - 4s 2ms/step - loss: 0.7177 - val_loss: 0.7005
Epoch 6/10
1948/1948 [==============================] - 4s 2ms/step - loss: 0.6875 - val_loss: 0.6710
Epoch 7/10
1948/1948 [==============================] - 4s 2ms/step - loss: 0.6585 - val_loss: 0.6426
Epoch 8/10
1948/1948 [==============================] - 4s 2ms/step - loss: 0.6305 - val_loss: 0.6151
Epoch 9/10
1948/1948 [==============================] - 4s 2ms/step - loss: 0.6035 - val_loss: 0.5887
Epoch 10/10
1948/1948 [============

Testing, evaluating.

In [0]:
testing_set_predictions=autoencoder.predict(X_test)


We can alter threshold with the best possible value for best results. Here threshold corresponds to val_loss of the training phase. 

In [0]:
def calculate_losses(x,preds):
    losses=np.zeros(len(x))
    for i in range(len(x)):
        losses[i]=((preds[i] - x[i]) ** 2).mean(axis=None)
        
    return losses

# We set the threshold equal to the training loss of the autoencoder
# history.history["loss"][-1] -- > Loss obtained in the last epoch
threshold=history.history["loss"][-4]


# Make predictions
testing_set_predictions=autoencoder.predict(X_test)

# Get the test loss
test_losses=calculate_losses(X_test,testing_set_predictions)

# Compare with the thresshold and assign the label/mark it as attack or normal
testing_set_predictions=np.zeros(len(test_losses))
testing_set_predictions[np.where(test_losses>threshold)]=1

Evaluating the model.

In [126]:
classes=[0,1]

accuracy=accuracy_score(y_test,testing_set_predictions)
recall=recall_score(y_test,testing_set_predictions,average='weighted')
precision=precision_score(y_test,testing_set_predictions,average='weighted')
f1=f1_score(y_test,testing_set_predictions,average='weighted')
print("Performance over the testing data set \n")
print("Accuracy : {} , Recall : {} , Precision : {} , F1 : {}\n".format(accuracy,recall,precision,f1 ))

#
for class_ in classes:
    print(class_," Detection Rate : {}".format(len(np.where(np.logical_and(testing_set_predictions==1, y_test==class_))[0])/len(np.where(y_test==class_)[0])))

Performance over the testing data set 

Accuracy : 0.9380378657487092 , Recall : 0.9380378657487092 , Precision : 0.8799150375783933 , F1 : 0.9080473123144698

0  Detection Rate : 0.0
1  Detection Rate : 0.0


In [79]:
np.where(testing_set_predictions==1)

(array([], dtype=int64),)

In [80]:
np.where(y_test==1)

(array([  25,   62,   69,   92,  101,  130,  138,  141,  186,  192,  196,
         204,  205,  218,  248,  262,  264,  270,  293,  296,  300,  336,
         337,  359,  373,  393,  413,  430,  463,  482,  496,  508,  523,
         535,  545,  575,  616,  623,  626,  671,  680,  720,  721,  727,
         730,  740,  766,  795,  804,  805,  806,  820,  836,  839,  846,
         850,  876,  891,  937,  949,  957,  965,  972,  987,  994, 1003,
        1020, 1022, 1026, 1030, 1032, 1044, 1053, 1065, 1071, 1079, 1087,
        1090, 1095]),)

In [68]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(y_test,testing_set_predictions)

-0.037546831972145846